<a href="https://colab.research.google.com/github/jbsdoki/Squishy_Robots_Quant_Models/blob/main/Squish_Robot_Quant_Model_3_VideoGasNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Create a study object and specify the direction of optimization (maximize accuracy)
study = optuna.create_study(direction='maximize',
                             pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5))

# Run the optimization
study.optimize(objective, n_trials = 30)

# Print the best hyperparameters found
print("Best hyperparameters: ", study.best_params)

# Print the best accuracy found
print("Best accuracy: ", study.best_value)

# Plot the visualization
optuna.visualization.plot_param_importances(study).show()

# Run more trials
# study.optimize(objective, n_trials=20)

[I 2025-10-30 20:43:11,217] A new study created in memory with name: no-name-b8de5c28-d97f-4179-88e7-3f4346da17db



Trial 0 | lr=0.000180 | optimizer=RMSprop | batch=64 | hidden=96
Epoch [ 1/20 ] Train Acc: 0.4799
Epoch [ 2/20 ] Train Acc: 0.5994
Epoch [ 3/20 ] Train Acc: 0.6625
Epoch [ 4/20 ] Train Acc: 0.7024
Epoch [ 5/20 ] Train Acc: 0.7317
Epoch [ 6/20 ] Train Acc: 0.7539
Epoch [ 7/20 ] Train Acc: 0.7716
Epoch [ 8/20 ] Train Acc: 0.7858
Epoch [ 9/20 ] Train Acc: 0.7978
Epoch [10/20 ] Train Acc: 0.8083
Epoch [11/20 ] Train Acc: 0.8172
Epoch [12/20 ] Train Acc: 0.8249
Epoch [13/20 ] Train Acc: 0.8321
Epoch [14/20 ] Train Acc: 0.8385
Epoch [15/20 ] Train Acc: 0.8445
Epoch [16/20 ] Train Acc: 0.8499
Epoch [17/20 ] Train Acc: 0.8548
Epoch [18/20 ] Train Acc: 0.8595
Epoch [19/20 ] Train Acc: 0.8637
Epoch [20/20 ] Train Acc: 0.8677


[I 2025-10-30 22:08:25,777] Trial 0 finished with value: 0.24268689057421453 and parameters: {'lr': 0.0001801884055750107, 'optimizer': 'RMSprop', 'momentum': 0.29337873784424356, 'weight_decay': 0.0016282381603796759, 'hidden_size': 96, 'batch_size': 64, 'num_epochs': 20}. Best is trial 0 with value: 0.24268689057421453.


Validation Acc: 0.2427


Trial 1 | lr=0.000078 | optimizer=SGD | batch=64 | hidden=218


#Sources:
###Hyperparameter Tuning with Optuna:
https://medium.com/@taeefnajib/hyperparameter-tuning-using-optuna-c46d7b29a3e

https://optuna.org/#code_examples
###Multi-Modal ML Models
https://www.nature.com/articles/s41598-025-14901-4

###Next Models to test:
VideoGasNet:
https://www.sciencedirect.com/science/article/pii/S0360544221017643

GasVit: https://www.sciencedirect.com/science/article/pii/S1568494623011560?via%3Dihub#sec3

In [1]:
pip install optuna #Hyperparameter Optimizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 16.8 MB/s eta 0:00:00


In [2]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from collections import defaultdict
from collections import Counter

from sklearn.model_selection import train_test_split

import optuna


In [3]:
!unzip -q Final_Dataset.zip

## Print out the structure of the data

In [4]:
# Example of loading the first file
file_path = './Final_Dataset/data/class_0/1237_frame_1004_class_0.npy'
sample_data = np.load(file_path)
print(f"Shape of preprocessed sample data: {sample_data.shape}")
print(f"Data type of preprocessed sample data: {sample_data.dtype}")

# GasVid synthetic processed dataset should be 2 channels, 240x320 in dimension

Shape of preprocessed sample data: (2, 240, 320)
Data type of preprocessed sample data: float32


## Load and preprocess the data

In [5]:
# Assuming the data is in 'Final_Dataset/data' and class folders are named 'class_0' ... 'class_7'
data_dir = 'Final_Dataset/data'
classes = sorted(os.listdir(data_dir))
print(f"Classes: {classes}")

Classes: ['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7']


In [6]:
file_paths = []
labels = []

# Group by video ID only (not by video+class)
video_to_files = defaultdict(list)
for i, class_name in enumerate(classes):
    class_dir = os.path.join(data_dir, class_name)
    for file_name in os.listdir(class_dir):
        if file_name.endswith('.npy'):
            video_id = file_name.split('_')[0]
            video_to_files[video_id].append((os.path.join(class_dir, file_name), i))

print(f"Number of videos: {len(video_to_files)}")

Number of videos: 28


In [7]:

# Function to load and preprocess a single .npy file
def load_and_preprocess_npy(filepath):
    data = np.load(filepath)
    if data.dtype != np.float32:
        data = data.astype(np.float32)

    return data

# Normalize the data
def normalize_data(data):
    tiny_number = 1e-5
    mean = data.mean()
    std = data.std()

    data = (data - mean) / (std + tiny_number)

    return data

In [8]:
# Split by video IDs
video_ids = list(video_to_files.keys())
train_vids, test_vids = train_test_split(video_ids, test_size=0.2, random_state=42)

train_files, train_labels = [], []
test_files, test_labels = [], []

for vid in train_vids:
    for filepath, label in video_to_files[vid]:
        train_files.append(filepath)
        train_labels.append(label)

for vid in test_vids:
    for filepath, label in video_to_files[vid]:
        test_files.append(filepath)
        test_labels.append(label)

In [9]:
print(f"Training videos: {sorted(train_vids)}")
print(f"Testing videos:  {sorted(test_vids)}")

# Make sure no overlap
overlap = set(train_vids) & set(test_vids)
print(f"Video overlap: {overlap if overlap else 'None'}")

# Make sure classes are distributed
print(f"\nTraining classes: {dict(sorted(Counter(train_labels).items()))}")
print(f"Testing classes:  {dict(sorted(Counter(test_labels).items()))}")

# Make sure both sets have all 8 classes
train_classes = set(train_labels)
test_classes = set(test_labels)
missing_train = set(range(8)) - train_classes
missing_test = set(range(8)) - test_classes

if missing_train:
    print(f"WARNING: Training missing classes {missing_train}")
if missing_test:
    print(f"WARNING: Testing missing classes {missing_test}")
if not missing_train and not missing_test:
    print(f"Both train and test have all 8 classes")

Training videos: ['1237', '1238', '1239', '1241', '1242', '1468', '1469', '1471', '1472', '2559', '2560', '2562', '2563', '2564', '2566', '2569', '2571', '2578', '2579', '2580', '2581', '2583']
Testing videos:  ['1240', '1467', '1470', '2561', '2567', '2568']
Video overlap: None

Training classes: {0: 4245, 1: 4247, 2: 4234, 3: 4242, 4: 4220, 5: 4261, 6: 4228, 7: 4250}
Testing classes:  {0: 1150, 1: 1146, 2: 1159, 3: 1155, 4: 1162, 5: 1150, 6: 1152, 7: 1156}
Both train and test have all 8 classes


## Create a dataset and dataloader

In [10]:


class NpyDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        filepath = self.file_paths[idx]
        label = self.labels[idx]
        data = load_and_preprocess_npy(filepath) # Reuse the functions from the previous step
        data = normalize_data(data)

        if self.transform:
            data = self.transform(data)

        return torch.from_numpy(data), torch.tensor(label)


# # Create instances of datasets
train_dataset = NpyDataset(train_files, train_labels)
test_dataset = NpyDataset(test_files, test_labels)

## Define the CNN model

## Define the Optuna Objective Function

This function will be called by Optuna for each trial. It will:
1. Suggest hyperparameters using the trial object.
2. Build and train the CNN model with the suggested hyperparameters.
3. Evaluate the model on a validation set
4. Return the metric to minimize (loss) or maximize (accuracy).

In [11]:
def objective(trial):

    #############################
    # All Hyperparameters Tested
    #############################
    lr = trial.suggest_float('lr', 1e-5, 1e-1, log=True)
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD', 'Adadelta', 'AdamW'])
    momentum = trial.suggest_float('momentum', 0.0, 0.99) if optimizer_name in ['SGD', 'RMSprop'] else 0.0
    weight_decay = trial.suggest_float('weight_decay', 0.0, 0.01)
    hidden_size = trial.suggest_int('hidden_size', 64, 256)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    num_epochs = trial.suggest_int('num_epochs', 5, 30)

    #####################
    # Define the Model
    #####################
    class VideoGasNet(nn.Module):
        def __init__(self):
            super(VideoGasNet, self).__init__()

            self.conv1 = nn.Conv2d(2, 32, kernel_size=3, padding=1)
            self.relu1 = nn.ReLU()
            self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

            self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
            self.relu2 = nn.ReLU()
            self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

            self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
            self.relu3 = nn.ReLU()
            self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

            # self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
            # self.relu4 = nn.ReLU()
            # self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

            # Calculate flatten size from conv layers from input(240x320)
            # flatten_size = 128 * (240 // 16) * (320 // 16) # 2^4 = 8 use for every conv + relu + pool section

            # Calculate flatten size from conv layers from input(240x320)
            flatten_size = 128 * (240 // 8) * (320 // 8) # 2^3 = 8 use for every conv + relu + pool section
            self.fc1 = nn.Linear(flatten_size, hidden_size)
            self.relu4 = nn.ReLU()
            self.fc2 = nn.Linear(hidden_size, 8)

        def forward(self, x):
            # Convolutional Blocks
            x = self.pool1(self.relu1(self.conv1(x)))
            x = self.pool2(self.relu2(self.conv2(x)))
            x = self.pool3(self.relu3(self.conv3(x)))
            # Fully Connected Blocks (Neural Network)
            x = x.view(x.size(0), -1)  # flatten
            x = self.relu4(self.fc1(x))
            x = self.fc2(x)
            return x

    model = VideoGasNet()

    ###############################
    # Define optimizer
    ###############################
    if optimizer_name == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    elif optimizer_name == 'RMSprop':
        optimizer = optim.RMSprop(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    elif optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'AdamW':
        optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == 'Adadelta':
        optimizer = optim.Adadelta(model.parameters(), lr=lr, weight_decay=weight_decay)

    criterion = nn.CrossEntropyLoss()

    ##########################################
    # Create DataLoaders with trial batch_size
    ##########################################
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    ###############################
    # Train the model
    ###############################
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)


    print(f"\n{'='*70}")
    print(f"Trial {trial.number} | lr={lr:.6f} | optimizer={optimizer_name} | "
          f"batch={batch_size} | hidden={hidden_size}")
    print(f"{'='*70}")


    model.train()
    train_correct = 0
    train_total = 0
    for epoch in range(num_epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Calculate training accuracy
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()

        #Print out training during each epoch
        train_accuracy = train_correct / train_total

        # Print training accuracy for this epoch
        print(f"Epoch [{epoch+1:2d}/{num_epochs} ] Train Acc: {train_accuracy:.4f}")

    #####################
    # Evaluate the model
    #####################
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total

    print(f"Validation Acc: {accuracy:.4f}")
    print(f"{'='*70}\n")

    return accuracy


## Run the Optuna Study

Now we will create an Optuna study and run the optimization process.

In [ ]:
# Create a study object and specify the direction of optimization (maximize accuracy)
study = optuna.create_study(direction='maximize',
                             pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5))

# Run the optimization
study.optimize(objective, n_trials = 30)

# Print the best hyperparameters found
print("Best hyperparameters: ", study.best_params)

# Print the best accuracy found
print("Best accuracy: ", study.best_value)

# Plot the visualization
optuna.visualization.plot_param_importances(study).show()

# Run more trials
# study.optimize(objective, n_trials=20)

[I 2025-10-30 20:43:11,217] A new study created in memory with name: no-name-b8de5c28-d97f-4179-88e7-3f4346da17db



Trial 0 | lr=0.000180 | optimizer=RMSprop | batch=64 | hidden=96
Epoch [ 1/20 ] Train Acc: 0.4799
Epoch [ 2/20 ] Train Acc: 0.5994
Epoch [ 3/20 ] Train Acc: 0.6625
Epoch [ 4/20 ] Train Acc: 0.7024
Epoch [ 5/20 ] Train Acc: 0.7317
Epoch [ 6/20 ] Train Acc: 0.7539
Epoch [ 7/20 ] Train Acc: 0.7716
Epoch [ 8/20 ] Train Acc: 0.7858
Epoch [ 9/20 ] Train Acc: 0.7978
Epoch [10/20 ] Train Acc: 0.8083
Epoch [11/20 ] Train Acc: 0.8172
Epoch [12/20 ] Train Acc: 0.8249
Epoch [13/20 ] Train Acc: 0.8321
Epoch [14/20 ] Train Acc: 0.8385
Epoch [15/20 ] Train Acc: 0.8445
Epoch [16/20 ] Train Acc: 0.8499
Epoch [17/20 ] Train Acc: 0.8548
Epoch [18/20 ] Train Acc: 0.8595
Epoch [19/20 ] Train Acc: 0.8637
Epoch [20/20 ] Train Acc: 0.8677


[I 2025-10-30 22:08:25,777] Trial 0 finished with value: 0.24268689057421453 and parameters: {'lr': 0.0001801884055750107, 'optimizer': 'RMSprop', 'momentum': 0.29337873784424356, 'weight_decay': 0.0016282381603796759, 'hidden_size': 96, 'batch_size': 64, 'num_epochs': 20}. Best is trial 0 with value: 0.24268689057421453.


Validation Acc: 0.2427


Trial 1 | lr=0.000078 | optimizer=SGD | batch=64 | hidden=218
